In [ ]:
!pip install gnews requests textblob scikit-learn nltk
!python -m textblob.download_corpora

  Using cached gnews-0.4.2-py3-none-any.whl.metadata (19 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 7.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=87d0203c08125c2ccf9bb9e8abb55082c93ab043e119ff06039378141c03f35c
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nl

In [2]:
import requests
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Replace with your GNews API key (get it from https://gnews.io/)
API_KEY = "8f0b1d8e2e0ded9d8c6e2d272bb4c523"

def fetch_news(query, max_articles=5):
    """Fetch news articles related to the query"""
    url = f"https://gnews.io/api/v4/search?q={query}&token={API_KEY}&lang=en&max={max_articles}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get("articles", [])
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return []

def analyze_sentiment(text):
    """Returns sentiment polarity (-1 to 1)"""
    return TextBlob(text).sentiment.polarity

def find_most_similar(target_title, articles):
    """Rank articles by title similarity to target"""
    titles = [article["title"] for article in articles]
    vectorizer = TfidfVectorizer().fit_transform([target_title] + titles)
    similarity_scores = cosine_similarity(vectorizer[0:1], vectorizer[1:]).flatten()
    return sorted(zip(articles, similarity_scores), key=lambda x: x[1], reverse=True)

def display_news(articles):
    """Print news with sentiment and similarity"""
    for idx, (article, score) in enumerate(articles, 1):
        sentiment = analyze_sentiment(article["title"])
        print(f"\n📰 Article {idx} (Similarity: {score:.2f}, Sentiment: {'👍' if sentiment > 0 else '👎' if sentiment < 0 else '😐'})")
        print(f"Title: {article['title']}")
        print(f"Source: {article['source']['name']}")
        print(f"URL: {article['url']}")
        print(f"Published: {article['publishedAt']}")
        print(f"Description: {article['description'][:200]}...")

# User input
user_query = input("Enter a news title or topic: ")
articles = fetch_news(user_query)

if articles:
    ranked_articles = find_most_similar(user_query, articles)
    display_news(ranked_articles)
else:
    print("No articles found. Try another query.")

Enter a news title or topic: Renewable energy advancements

📰 Article 1 (Similarity: 0.25, Sentiment: 😐)
Title: Ladakh's Development Journey: Renewable Energy, Infrastructure, and Tourism
Source: Devdiscourse
URL: https://www.devdiscourse.com/article/science-environment/3736054-ladakhs-development-journey-renewable-energy-infrastructure-and-tourism
Published: 2025-12-19T11:45:07Z
Description: The Lieutenant Governor of Ladakh, Kavinder Gupta, discussed ongoing efforts for the region's development, highlighting proposals for statehood, infrastructure projects, and renewable energy advanceme...

📰 Article 2 (Similarity: 0.18, Sentiment: 😐)
Title: Renewable energy adoption rises in Indian textile sector, but intensity-related challenges persist: Report
Source: The Economic Times
URL: https://economictimes.indiatimes.com/industry/renewables/renewable-energy-adoption-rises-in-indian-textile-sector-but-intensity-related-challenges-persist-report/articleshow/125929392.cms
Published: 2025-12-1